<a href="https://colab.research.google.com/github/Benjamindavid03/MachineLearningLab/blob/main/Linear_Regression_using_pySpark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://downloads.apache.org/spark/spark-3.0.3/spark-3.0.3-bin-hadoop2.7.tgz
!tar xf spark-3.0.3-bin-hadoop2.7.tgz
!pip install -q findspark

import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.3-bin-hadoop2.7"
!export SPARK_HOME=/content/spark-3.0.3-bin-hadoop2.7
! echo $SPARK_HOME

/content/spark-3.0.3-bin-hadoop2.7


In [3]:
import findspark
findspark.init()

In [ ]:
import pyspark
from pyspark.sql import SparkSession
#SparkSession is now the entry point of Spark
#SparkSession can also be construed as gateway to spark libraries
  
#create instance of spark class
spark=SparkSession.builder.appName('housing_price_model').getOrCreate()
  

from google.colab import drive
drive.mount('/content/drive')

#create spark dataframe of input csv file
df=spark.read.csv("/content/drive/MyDrive/Colab Datasets/cruise_ship_info.csv",inferSchema=True,header=True)
df.show(10)

In [ ]:
#prints structure of dataframe along with datatype
df.printSchema()
#In our predictive model, below are the columns
df.columns

In [11]:
#columns identified as features are as below:
#['Cruise_line','Age','Tonnage','passengers','length','cabins','passenger_density']
#to work on the features, spark MLlib expects every value to be in numeric form
#feature 'Cruise_line is string datatype
#using StringIndexer, string type will be typecast to numeric datatype
#import library strinindexer for typecasting
  
from pyspark.ml.feature import StringIndexer
indexer=StringIndexer(inputCol='Cruise_line',outputCol='cruise_cat')
indexed=indexer.fit(df).transform(df)
  
#above code will convert string to numeric feature and create a new dataframe
#new dataframe contains a new feature 'cruise_cat' and can be used further
#feature cruise_cat is now vectorized and can be used to fed to model
for item in indexed.head(5):
    print(item)
    print('\n')

Row(Ship_name='Journey', Cruise_line='Azamara', Age=6, Tonnage=30.276999999999997, passengers=6.94, length=5.94, cabins=3.55, passenger_density=42.64, crew=3.55, cruise_cat=16.0)


Row(Ship_name='Quest', Cruise_line='Azamara', Age=6, Tonnage=30.276999999999997, passengers=6.94, length=5.94, cabins=3.55, passenger_density=42.64, crew=3.55, cruise_cat=16.0)


Row(Ship_name='Celebration', Cruise_line='Carnival', Age=26, Tonnage=47.262, passengers=14.86, length=7.22, cabins=7.43, passenger_density=31.8, crew=6.7, cruise_cat=1.0)


Row(Ship_name='Conquest', Cruise_line='Carnival', Age=11, Tonnage=110.0, passengers=29.74, length=9.53, cabins=14.88, passenger_density=36.99, crew=19.1, cruise_cat=1.0)


Row(Ship_name='Destiny', Cruise_line='Carnival', Age=17, Tonnage=101.353, passengers=26.42, length=8.92, cabins=13.21, passenger_density=38.36, crew=10.0, cruise_cat=1.0)




In [12]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler
#creating vectors from features
#Apache MLlib takes input if vector form
assembler=VectorAssembler(inputCols=['Age',
 'Tonnage',
 'passengers',
 'length',
 'cabins',
 'passenger_density',
 'cruise_cat'],outputCol='features')
output=assembler.transform(indexed)
output.select('features','crew').show(5)
#output as below

+--------------------+----+
|            features|crew|
+--------------------+----+
|[6.0,30.276999999...|3.55|
|[6.0,30.276999999...|3.55|
|[26.0,47.262,14.8...| 6.7|
|[11.0,110.0,29.74...|19.1|
|[17.0,101.353,26....|10.0|
+--------------------+----+
only showing top 5 rows



In [13]:
#final data consist of features and label which is crew.
final_data=output.select('features','crew')
#splitting data into train and test
train_data,test_data=final_data.randomSplit([0.7,0.3])
train_data.describe().show()

+-------+------------------+
|summary|              crew|
+-------+------------------+
|  count|               118|
|   mean| 7.670084745762723|
| stddev|3.2369301949183056|
|    min|              0.59|
|    max|              13.6|
+-------+------------------+



In [14]:
test_data.describe().show()

+-------+-----------------+
|summary|             crew|
+-------+-----------------+
|  count|               40|
|   mean|8.160250000000001|
| stddev|4.218437241018321|
|    min|              1.6|
|    max|             21.0|
+-------+-----------------+



In [16]:
#import LinearRegression library
from pyspark.ml.regression import LinearRegression
#creating an object of class LinearRegression
#object takes features and label as input arguments
ship_lr=LinearRegression(featuresCol='features',labelCol='crew')
#pass train_data to train model
trained_ship_model=ship_lr.fit(train_data)
#evaluating model trained for Rsquared error
ship_results=trained_ship_model.evaluate(train_data)
  
print('Rsquared Error :',ship_results.r2)
#R2 value shows accuracy of model is 92%
#model accuracy is very good and can be use for predictive analysis

Rsquared Error : 0.9394962587790356


In [17]:
#testing Model on unlabeled data
#create unlabeled data from test_data
#testing model on unlabeled data
unlabeled_data=test_data.select('features')
unlabeled_data.show(5)

+--------------------+
|            features|
+--------------------+
|[4.0,220.0,54.0,1...|
|[6.0,30.276999999...|
|[6.0,110.23899999...|
|[6.0,112.0,38.0,9...|
|[6.0,158.0,43.7,1...|
+--------------------+
only showing top 5 rows



In [18]:
predictions=trained_ship_model.transform(unlabeled_data)
predictions.show()
#below are the results of output from test data

+--------------------+------------------+
|            features|        prediction|
+--------------------+------------------+
|[4.0,220.0,54.0,1...|20.644697039328424|
|[6.0,30.276999999...| 4.261278214183823|
|[6.0,110.23899999...|10.922799112386892|
|[6.0,112.0,38.0,9...|11.115945491396499|
|[6.0,158.0,43.7,1...|13.845228080328125|
|[8.0,77.499,19.5,...|   8.4644993079824|
|[9.0,59.058,17.0,...| 7.435461802291096|
|[10.0,77.0,20.16,...| 8.668777113946716|
|[10.0,90.09,25.01...| 8.829325021568703|
|[11.0,86.0,21.24,...| 9.403638332830656|
|[11.0,90.0,22.4,9...| 9.919131079370606|
|[11.0,91.0,20.32,...| 9.183777402456958|
|[11.0,110.0,29.74...|11.902711878314337|
|[11.0,138.0,31.14...|12.908479452405654|
|[12.0,25.0,3.88,5...| 2.966691620780472|
|[12.0,50.0,7.0,7....| 4.556563541016106|
|[12.0,58.6,15.66,...| 7.317272557410638|
|[12.0,77.104,20.0...|  8.67259975811991|
|[12.0,88.5,21.24,...| 9.392858711985015|
|[12.0,91.0,20.32,...| 9.181775089762116|
+--------------------+------------